<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h1 style="padding: 10px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Transformacija podataka</h1>
</div>

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h2 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Prikupljanje i predobrada podataka za Mašinsko učenje</h2>
</div>

<div style="border-radius:10px;
            background-color:#D6EAF8;
            letter-spacing:0.5px; text-align: center;">
            
<h2 style="padding: 5px 0px; color:#2980B9; font-weight: bold; font-family: Arial, sans-serif;">
Učitavanje biblioteka</h2>
</div>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

import warnings

In [2]:
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler

In [3]:
pd.set_option('display.max_columns', None)
df = pd.read_csv("vehiclesData.csv", sep=",")
df.head(100)

warnings.filterwarnings('ignore')
np.random.seed(80)

In [4]:
df.head(10)

,timestep,id,x,y,angle,type,speed,pos,lane,slope,signals,acceleration,distance,odometer,posLat
0,0,bus0,-104.996667,39.732112,270.14,bus_bus,0.00,12.10,336167361#0_0,0,0,0.00,12.10,0.00,0
1,0,veh0,-104.990763,39.725715,90.39,veh_passenger,0.00,5.10,427704888#4_4,0,0,0.00,5.10,0.00,0
2,1,bus0,-104.996674,39.732112,270.12,bus_bus,0.62,12.72,336167361#0_0,0,0,0.62,12.72,0.62,0
3,1,veh0,-104.990736,39.725715,90.39,veh_passenger,2.34,7.44,427704888#4_4,0,2,2.34,7.44,2.34,0
4,1,veh1,-104.990296,39.729562,328.88,veh_passenger,0.00,5.10,1121174297_1,0,0,0.00,5.10,0.00,0
5,1,veh2,-104.960328,39.727314,269.85,veh_passenger,0.00,5.10,586800346#3_0,0,0,0.00,5.10,0.00,0
6,2,bus0,-104.996692,39.732112,270.10,bus_bus,1.57,14.29,336167361#0_0,0,0,0.95,14.29,2.19,0
7,2,bus1,-105.024910,39.747560,89.86,bus_bus,0.00,12.10,-628592111#3_0,0,0,0.00,12.10,0.00,0
8,2,veh0,-104.990692,39.725715,90.39,veh_passenger,3.78,11.22,427704888#4_4,0,2,1.44,11.22,6.12,0
9,2,veh1,-104.990303,39.729573,329.73,veh_passenger,1.38,6.48,1121174297_1,0,0,1.38,6.48,1.38,0


In [5]:
df = df.drop(['angle', 'pos', 'lane', 'slope', 'signals', 'posLat'], axis=1)\
       .rename(columns={'x': 'longitude', 'y': 'latitude'})\
       .assign(speed_kmh=lambda x: round(x['speed'] * 3.6, 2))\
       .drop(['speed'],axis=1)


In [6]:
df.dtypes

timestep          int64
id               object
longitude       float64
latitude        float64
type             object
acceleration    float64
distance        float64
odometer        float64
speed_kmh       float64
dtype: object

In [7]:
df.head(10)

,timestep,id,longitude,latitude,type,acceleration,distance,odometer,speed_kmh
0,0,bus0,-104.996667,39.732112,bus_bus,0.00,12.10,0.00,0.00
1,0,veh0,-104.990763,39.725715,veh_passenger,0.00,5.10,0.00,0.00
2,1,bus0,-104.996674,39.732112,bus_bus,0.62,12.72,0.62,2.23
3,1,veh0,-104.990736,39.725715,veh_passenger,2.34,7.44,2.34,8.42
4,1,veh1,-104.990296,39.729562,veh_passenger,0.00,5.10,0.00,0.00
5,1,veh2,-104.960328,39.727314,veh_passenger,0.00,5.10,0.00,0.00
6,2,bus0,-104.996692,39.732112,bus_bus,0.95,14.29,2.19,5.65
7,2,bus1,-105.024910,39.747560,bus_bus,0.00,12.10,0.00,0.00
8,2,veh0,-104.990692,39.725715,veh_passenger,1.44,11.22,6.12,13.61
9,2,veh1,-104.990303,39.729573,veh_passenger,1.38,6.48,1.38,4.97


In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
timestep,18952.0,377.691800,278.832309,0.000000,56.000000,404.000000,408.000000,910.000000
longitude,18952.0,-104.991964,0.015416,-105.034328,-105.002608,-104.990640,-104.981536,-104.958775
latitude,18952.0,39.740986,0.007822,39.716626,39.735347,39.741552,39.747503,39.765507
acceleration,18952.0,0.054794,1.614122,-4.500000,-0.210000,0.010000,1.020000,2.600000
distance,18952.0,69.404090,70.534907,0.000000,23.970000,55.120000,91.345000,916.740000
odometer,18952.0,1226.539942,1191.130501,0.000000,202.060000,854.485000,2048.800000,7546.980000
speed_kmh,18952.0,30.533046,24.435111,0.000000,5.180000,31.430000,47.480000,114.160000


In [9]:
df = df[['id','timestep','type','latitude','longitude','speed_kmh','acceleration','distance','odometer']]

In [10]:
df.head(10)

,id,timestep,type,latitude,longitude,speed_kmh,acceleration,distance,odometer
0,bus0,0,bus_bus,39.732112,-104.996667,0.00,0.00,12.10,0.00
1,veh0,0,veh_passenger,39.725715,-104.990763,0.00,0.00,5.10,0.00
2,bus0,1,bus_bus,39.732112,-104.996674,2.23,0.62,12.72,0.62
3,veh0,1,veh_passenger,39.725715,-104.990736,8.42,2.34,7.44,2.34
4,veh1,1,veh_passenger,39.729562,-104.990296,0.00,0.00,5.10,0.00
5,veh2,1,veh_passenger,39.727314,-104.960328,0.00,0.00,5.10,0.00
6,bus0,2,bus_bus,39.732112,-104.996692,5.65,0.95,14.29,2.19
7,bus1,2,bus_bus,39.747560,-105.024910,0.00,0.00,12.10,0.00
8,veh0,2,veh_passenger,39.725715,-104.990692,13.61,1.44,11.22,6.12
9,veh1,2,veh_passenger,39.729573,-104.990303,4.97,1.38,6.48,1.38


In [11]:
df['type'] = df['type'].replace({'bus_bus':'bus', 'veh_passenger':'car'})


In [12]:
df['timestep'].nunique()

75

In [13]:
df['time_of_day'] = df['timestep'].apply(lambda x: 'morning' if x < 61 else ('afternoon' if 61 <= x <= 899 else 'evening'))
df.drop(['timestep','id'],axis=1)

,type,latitude,longitude,speed_kmh,acceleration,distance,odometer,time_of_day
0,bus,39.732112,-104.996667,0.00,0.00,12.10,0.00,morning
1,car,39.725715,-104.990763,0.00,0.00,5.10,0.00,morning
2,bus,39.732112,-104.996674,2.23,0.62,12.72,0.62,morning
3,car,39.725715,-104.990736,8.42,2.34,7.44,2.34,morning
4,car,39.729562,-104.990296,0.00,0.00,5.10,0.00,morning
...,...,...,...,...,...,...,...,...
18947,car,39.740587,-104.992876,56.59,-4.50,102.64,2142.57,evening
18948,car,39.754406,-104.985458,68.98,-0.58,92.83,2811.17,evening
18949,car,39.748118,-104.986578,24.05,-1.64,77.25,2318.51,evening
18950,car,39.727299,-104.999915,29.09,1.71,12.38,3416.51,evening


In [14]:
df.dtypes

id               object
timestep          int64
type             object
latitude        float64
longitude       float64
speed_kmh       float64
acceleration    float64
distance        float64
odometer        float64
time_of_day      object
dtype: object

In [15]:
# provlacenje kategorickih atributa kroz one hot encoder
cols = ['type', 'time_of_day']
konacni_df = pd.get_dummies(data=df, columns=cols)
konacni_df.head()

,id,timestep,latitude,longitude,speed_kmh,acceleration,distance,odometer,type_bus,type_car,time_of_day_afternoon,time_of_day_evening,time_of_day_morning
0,bus0,0,39.732112,-104.996667,0.00,0.00,12.10,0.00,1,0,0,0,1
1,veh0,0,39.725715,-104.990763,0.00,0.00,5.10,0.00,0,1,0,0,1
2,bus0,1,39.732112,-104.996674,2.23,0.62,12.72,0.62,1,0,0,0,1
3,veh0,1,39.725715,-104.990736,8.42,2.34,7.44,2.34,0,1,0,0,1
4,veh1,1,39.729562,-104.990296,0.00,0.00,5.10,0.00,0,1,0,0,1


In [16]:
konacni_df.drop(['id','timestep'],axis=1,inplace=True)

In [17]:
df_normalized_minmax = konacni_df.copy()
df_normalized_maxabs = konacni_df.copy()
df_normalized_robust = konacni_df.copy()
df_standardized = konacni_df.copy()

In [18]:
konacni_df.head(20)

,latitude,longitude,speed_kmh,acceleration,distance,odometer,type_bus,type_car,time_of_day_afternoon,time_of_day_evening,time_of_day_morning
0,39.732112,-104.996667,0.00,0.00,12.10,0.00,1,0,0,0,1
1,39.725715,-104.990763,0.00,0.00,5.10,0.00,0,1,0,0,1
2,39.732112,-104.996674,2.23,0.62,12.72,0.62,1,0,0,0,1
3,39.725715,-104.990736,8.42,2.34,7.44,2.34,0,1,0,0,1
4,39.729562,-104.990296,0.00,0.00,5.10,0.00,0,1,0,0,1
5,39.727314,-104.960328,0.00,0.00,5.10,0.00,0,1,0,0,1
6,39.732112,-104.996692,5.65,0.95,14.29,2.19,1,0,0,0,1
7,39.747560,-105.024910,0.00,0.00,12.10,0.00,1,0,0,0,1
8,39.725715,-104.990692,13.61,1.44,11.22,6.12,0,1,0,0,1
9,39.729573,-104.990303,4.97,1.38,6.48,1.38,0,1,0,0,1


In [20]:
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import RobustScaler

scalerStd = StandardScaler()
scalerMinMax = MinMaxScaler()
scalerMaxAbs = MaxAbsScaler()
scalerRobust = RobustScaler()
df_normalized_minmax = pd.DataFrame(scalerMinMax.fit_transform(df_normalized_minmax), columns = df_normalized_minmax.columns)
df_normalized_maxabs = pd.DataFrame(scalerMaxAbs.fit_transform(df_normalized_maxabs), columns = df_normalized_maxabs.columns)
df_normalized_robust = pd.DataFrame(scalerRobust.fit_transform(df_normalized_robust), columns = df_normalized_robust.columns)


df_standardized = pd.DataFrame(scalerStd.fit_transform(df_standardized), columns = df_standardized.columns)

In [21]:
df_standardized.head(n=5)

,latitude,longitude,speed_kmh,acceleration,distance,odometer,type_bus,type_car,time_of_day_afternoon,time_of_day_evening,time_of_day_morning
0,-1.134599,-0.305067,-1.249589,-0.033947,-0.812443,-1.029755,1.625581,-1.625581,-1.111382,-0.43074,1.561311
1,-1.952471,0.077928,-1.249589,-0.033947,-0.911687,-1.029755,-0.615165,0.615165,-1.111382,-0.43074,1.561311
2,-1.134599,-0.305521,-1.158325,0.350173,-0.803653,-1.029234,1.625581,-1.625581,-1.111382,-0.43074,1.561311
3,-1.952471,0.079679,-0.904994,1.415796,-0.878511,-1.027790,-0.615165,0.615165,-1.111382,-0.43074,1.561311
4,-1.460623,0.108222,-1.249589,-0.033947,-0.911687,-1.029755,-0.615165,0.615165,-1.111382,-0.43074,1.561311


In [22]:
df_normalized_minmax.head(n=5)

,latitude,longitude,speed_kmh,acceleration,distance,odometer,type_bus,type_car,time_of_day_afternoon,time_of_day_evening,time_of_day_morning
0,0.316810,0.498471,0.000000,0.633803,0.013199,0.000000,1.0,0.0,0.0,0.0,1.0
1,0.185941,0.576615,0.000000,0.633803,0.005563,0.000000,0.0,1.0,0.0,0.0,1.0
2,0.316810,0.498379,0.019534,0.721127,0.013875,0.000082,1.0,0.0,0.0,0.0,1.0
3,0.185941,0.576972,0.073756,0.963380,0.008116,0.000310,0.0,1.0,0.0,0.0,1.0
4,0.264643,0.582796,0.000000,0.633803,0.005563,0.000000,0.0,1.0,0.0,0.0,1.0


In [23]:
df_normalized_maxabs.head(n=5)

,latitude,longitude,speed_kmh,acceleration,distance,odometer,type_bus,type_car,time_of_day_afternoon,time_of_day_evening,time_of_day_morning
0,0.999160,-0.999641,0.000000,0.000000,0.013199,0.000000,1.0,0.0,0.0,0.0,1.0
1,0.998999,-0.999585,0.000000,0.000000,0.005563,0.000000,0.0,1.0,0.0,0.0,1.0
2,0.999160,-0.999642,0.019534,0.137778,0.013875,0.000082,1.0,0.0,0.0,0.0,1.0
3,0.998999,-0.999585,0.073756,0.520000,0.008116,0.000310,0.0,1.0,0.0,0.0,1.0
4,0.999096,-0.999581,0.000000,0.000000,0.005563,0.000000,0.0,1.0,0.0,0.0,1.0


In [24]:
df_normalized_robust.head(n=5)

,latitude,longitude,speed_kmh,acceleration,distance,odometer,type_bus,type_car,time_of_day_afternoon,time_of_day_evening,time_of_day_morning
0,-0.776571,-0.286016,-0.743026,-0.008130,-0.638516,-0.462699,1.0,-1.0,-1.0,0.0,1.0
1,-1.302813,-0.005837,-0.743026,-0.008130,-0.742412,-0.462699,0.0,0.0,-1.0,0.0,1.0
2,-0.776571,-0.286348,-0.690307,0.495935,-0.629314,-0.462363,1.0,-1.0,-1.0,0.0,1.0
3,-1.302813,-0.004556,-0.543972,1.894309,-0.707681,-0.461432,0.0,0.0,-1.0,0.0,1.0
4,-0.986344,0.016325,-0.743026,-0.008130,-0.742412,-0.462699,0.0,0.0,-1.0,0.0,1.0
